### Towards General State Transitons with Cython

Just playing a little bit with Cython here, the case follows below.

In [1]:
%load_ext Cython

In [2]:
%%cython
cdef int a = 0
for i in range(10):
    a += i
print(a)

Content of stdout:
_cython_magic_881774f5de1efe2eb67e98c51cde4bf4f1c0f274.c
   Creating library C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_881774f5de1efe2eb67e98c51cde4bf4f1c0f274.cp39-win_amd64.lib and object C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_881774f5de1efe2eb67e98c51cde4bf4f1c0f274.cp39-win_amd64.exp
Generating code
Finished generating code45


In [3]:
%%cython --annotate -c infer_types
import cython

@cython.cfunc
def f(x: cython.double) -> cython.double:
    return x ** 2 - x


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def integrate_f(a: cython.double, b: cython.double, N: cython.int):
    i: cython.int
    s: cython.double
    dx: cython.double
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Content of stdout:
_cython_magic_2d77348bed198f6ca0985d5742e7b36e6e014dd3.c
   Creating library C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_2d77348bed198f6ca0985d5742e7b36e6e014dd3.cp39-win_amd64.lib and object C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_2d77348bed198f6ca0985d5742e7b36e6e014dd3.cp39-win_amd64.exp
Generating code
Finished generating codeContent of stderr:
cl : Command line warning D9024 : unrecognized source file type 'infer_types', object file assumed
cl : Command line warning D9027 : source file 'infer_types' ignored

In [4]:
integrate_f(0, 1, 20)

-0.16625

In [5]:
%%cython -+ --annotate -c infer_types
import cython

@cython.cfunc
def f(x: cython.double) -> cython.double:
    return x ** 2 - x

@cython.cfunc
@cython.cdivision(True)
def integrate_f_cfunc(a: cython.double, b: cython.double, N: cython.int) -> cython.double:
    i: cython.int
    s: cython.double
    dx: cython.double
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

def integrate_f_cfunc_wrapper(a: cython.double, b: cython.double, N: cython.int):
    return integrate_f_cfunc(a, b, N)

Content of stdout:
_cython_magic_db35429a2218511b526a8a1531a22b29156371f9.cpp
   Creating library C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_db35429a2218511b526a8a1531a22b29156371f9.cp39-win_amd64.lib and object C:\Users\martin\.ipython\cython\Users\martin\.ipython\cython\_cython_magic_db35429a2218511b526a8a1531a22b29156371f9.cp39-win_amd64.exp
Generating code
Finished generating codeContent of stderr:
cl : Command line warning D9024 : unrecognized source file type 'infer_types', object file assumed
cl : Command line warning D9027 : source file 'infer_types' ignored

In [6]:
integrate_f_cfunc_wrapper(0, 1, 20)

-0.16625

### Matrix of Independent Yearly Decrements

We start by defining a matrix of decrements. Te elements on the diagonal are not relevant and are set to nil.

In [7]:
import numpy as np

Q = np.zeros((3, 3))
Q[0, 1] = 0.1
Q[0, 2] = 0.2
Q[1, 0] = 0.3
Q[1, 2] = 0.4
#Q[2, 0] = 0.5
#Q[2, 1] = 0.6
#Q[1, 0] = 0.0
#Q[1, 2] = 0.0
Q[2, 0] = 0.0
Q[2, 1] = 0.0

Q_backup = Q.copy()

Q

array([[0. , 0.1, 0.2],
       [0.3, 0. , 0.4],
       [0. , 0. , 0. ]])

Next transform into a *matrix of forces* of decrements.

In [8]:

A = -np.log(1 - Q).transpose()
ind_tmp = np.arange(len(A))
A[ind_tmp, ind_tmp ] = -A.sum(axis=0)
A

array([[-0.32850407,  0.35667494, -0.        ],
       [ 0.10536052, -0.86750057, -0.        ],
       [ 0.22314355,  0.51082562, -0.        ]])

In [9]:
# the final time required
tau = 0.5
N = 120
h = tau / N

start_value = np.eye(3)

In [10]:
it = start_value

# just for checking:
intermediate_states = np.zeros((N + 1, A.shape[0], A.shape[1]))
intermediate_states[0, :, :] = it

# apply Euler iteration to get the general solution
solution_integral = np.zeros(it.shape)

for k in range(N):
    # integrate with begin of step value
    solution_integral += it
    it += h * np.dot(A, it)
    intermediate_states[k + 1, :, :] = it

solution_integral *= h

it, solution_integral

(array([[0.85206979, 0.13312575, 0.        ],
        [0.03932488, 0.65089414, 0.        ],
        [0.10860533, 0.21598011, 1.        ]]),
 array([[0.46202207, 0.03650251, 0.        ],
        [0.01078271, 0.40686058, 0.        ],
        [0.02719522, 0.05663692, 0.5       ]]))

In [11]:
def integrate_general(tau, A):
    """ Integrate the ODE using fixed step width return the general solution. """
    
    h = 0.001
    N = tau / h
    #assert N is integral
    print(N)
    N = int(N)
    assert len(A.shape) == 2 and A.shape[0] == A.shape[1]
    
    # start value
    it = np.eye(A.shape[0])
    # apply Euler iteration to get the general solution
    solution_integral = np.zeros(it.shape)

    for k in range(N):
        # integrate with begin of step value
        solution_integral += it
        it += h * np.dot(A, it)
        # solution_integral += it * 0.5
        # intermediate_states[k + 1, :, :] = it

    solution_integral *= h

    return it, solution_integral

In [12]:
def integrate_special(tau, start_state, A):
    """ Integrate the ODE using fixed step width return the general solution. """
    
    h = 0.01
    N = tau / h
    #assert N is integral
    print(N)
    N = int(N)
    assert len(A.shape) == 2 and A.shape[0] == A.shape[1]
    
    # start value
    it = np.array(start_state, dtype=np.float64)
    
    # apply Euler iteration to get the general solution
    solution_integral = np.zeros(it.shape)

    for k in range(N):
        # integrate with begin of step value
        solution_integral += it
        it += h * np.dot(A, it)
        # solution_integral += it * 0.5
        # intermediate_states[k + 1, :, :] = it

    solution_integral *= h

    return it, solution_integral

In [13]:
# let's compare general and special solution
start_state = np.array([1, 0, 0])
sol_spec, int_spec = integrate_special(tau=1.0, start_state=start_state, A=A)
sol_spec, int_spec

100.0


(array([0.73102025, 0.05923756, 0.20974219]),
 array([0.8577738 , 0.03589385, 0.10633235]))

In [14]:
sol, integral = integrate_general(tau=1.0, A=A)
sol.dot(start_state), integral.dot(start_state)

1000.0


(array([0.7313869 , 0.05903159, 0.20958151]),
 array([0.85678511, 0.03601119, 0.1072037 ]))

In [15]:
# check consistency over time
# we integrate twice for tau=0.5 is sequence
sol_spec05, int_spec05 = integrate_special(tau=0.5, start_state=start_state, A=A)
integrate_special(tau=0.5, start_state=sol_spec05, A=A)

50.0
50.0


(array([0.73102025, 0.05923756, 0.20974219]),
 array([0.39536257, 0.02519328, 0.07944415]))

In [16]:
# we see that the iterates state probabilities match

# moroever, the sum of the integrals of the two periods give the total integral, (the second half-perdiod one is just copied)
int_spec05, int_spec05 + np.array([0.39536257, 0.02519328, 0.07944415])

(array([0.46241123, 0.01070056, 0.0268882 ]),
 array([0.8577738 , 0.03589384, 0.10633235]))

In [17]:
# this show that we can calculate the "integrals" for subsequent special steps using
# the general solution
_, integral05 = integrate_general(tau=0.5, A=A)
integral05.dot(start_state), integral05.dot(sol_spec05)

500.0


(array([0.46181127, 0.01082701, 0.02736172]),
 array([0.39486833, 0.02525183, 0.07987984]))

In [18]:
# next look at probability transitions, start with the matrix of forces

prob_mov_05 = (A * integral05.dot(start_state)).transpose()
prob_mov_05

array([[-0.15170688,  0.04865667,  0.10305021],
       [ 0.00386172, -0.00939244,  0.00553071],
       [-0.        , -0.        , -0.        ]])

In [19]:
# can we obtain the state back with this?
state05 = start_state + start_state.dot(prob_mov_05)
state05

array([0.84829312, 0.04865667, 0.10305021])

In [20]:
sol_spec05

array([0.85191265, 0.03943714, 0.10865021])

 WHY NOT?

In [21]:
# this is the right way to do it: consider all movements
state05_v2 = start_state + prob_mov_05.sum(axis=0)
state05_v2

array([0.85215484, 0.03926424, 0.10858092])

In [22]:
# ok, and based on that iterate
prob_mov_05_v2 = (A * integral05.dot(state05_v2)).transpose()
state05_v2 + prob_mov_05_v2.sum(axis=0), sol_spec

(array([0.7313869 , 0.05903159, 0.20958151]),
 array([0.73102025, 0.05923756, 0.20974219]))

In [23]:

# next step should be a comparison against an exact solution
q = 0.01
r = 0.5

Q = np.array([[0, q, r], [0, 0, 0], [0, 0, 0]])
Q

array([[0.  , 0.01, 0.5 ],
       [0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  ]])

In [24]:
# redesign the function such that we can input Q and the number of steps
def integrate_general2(tau, Q, N=None):
    """ Integrate the ODE using fixed step width return the general solution. """
    
    assert len(Q.shape) == 2 and Q.shape[0] == Q.shape[1]
    
    # calculate a default step width
    if N is None:
        N = int(round(tau / 0.01))
        print("Steps used", N)

    A = -np.log(1 - Q).transpose()
    ind_tmp = np.arange(len(A))
    A[ind_tmp, ind_tmp ] = -A.sum(axis=0)
    
    h = tau / N

    # start value
    it = np.eye(A.shape[0])
    # apply Euler iteration to get the general solution
    solution_integral = np.zeros(it.shape)

    for k in range(N):
        # integrate with begin of step value
        solution_integral += it
        it += h * np.dot(A, it)
        # solution_integral += it * 0.5
        # intermediate_states[k + 1, :, :] = it

    solution_integral *= h

    return it, solution_integral

In [25]:
it, _ = integrate_general2(0.25, Q)
it

Steps used 25


array([[0.8382655 , 0.        , 0.        ],
       [0.00231156, 1.        , 0.        ],
       [0.15942293, 0.        , 1.        ]])

We had calculated this
for the exact solution: [0.83878624, 0.00230412, 0.15890963]

In [26]:
it100, _ = integrate_general2(0.25, Q, N=20000)
it100

array([[0.8387856 , 0.        , 0.        ],
       [0.00230413, 1.        , 0.        ],
       [0.15891027, 0.        , 1.        ]])

In [27]:
# redesign the function such that we can input Q and the number of steps
def integrate_general_heun(tau, Q, N=None):
    """ Integrate the ODE using Heun's approximation. """
    
    assert len(Q.shape) == 2 and Q.shape[0] == Q.shape[1]
    
    # calculate a default step width
    if N is None:
        N = 2 * int(round(tau / 0.01))
        print("Steps used", N)

    A = -np.log(1 - Q).transpose()
    ind_tmp = np.arange(len(A))
    A[ind_tmp, ind_tmp ] = -A.sum(axis=0)
    
    h = tau / N

    # start value
    it = np.eye(A.shape[0])
    
    # apply trapezoid rule for the numerical integral
    solution_integral = np.zeros(it.shape)
    solution_integral[:] = 0.5 * it

    for k in range(N):
        
        # Heun: Phi(x, y) = 0.5 * (f(x, y) + f(x+h, y+hf(x,y)))
        #                 = 0.5 * (Ay  +  A(y + h Ay))
        Ay = np.dot(A, it)
        z =  np.dot(A, it + h * Ay)
        it += h * 0.5 * (Ay + z)

        # add summand to integral
        solution_integral += (0.5 if k == N - 1 else 1) * it
        
        # solution_integral += it * 0.5
        # intermediate_states[k + 1, :, :] = it

    solution_integral *= h

    return it, solution_integral

In [28]:
it100, integral = integrate_general_heun(0.25, Q, N=100)
it100, integral  # check against:[0.83878624, 0.00230412, 0.15890963]

(array([[0.83878632, 0.        , 0.        ],
        [0.00230412, 1.        , 0.        ],
        [0.15890956, 0.        , 1.        ]]),
 array([[0.22925821, 0.        , 0.        ],
        [0.00029645, 0.25      , 0.        ],
        [0.02044534, 0.        , 0.25      ]]))

In [29]:
it100, integral = integrate_general2(0.25, A, N= 10000)
integral

array([[0.24528575, 0.00358688, 0.00739595],
       [0.01421114, 0.26688148, 0.02186108],
       [0.        , 0.        , 0.22206414]])

In [30]:
%%timeit
integrate_general_heun(0.25, Q, N=100)

833 µs ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
def integrate_general_rk(tau, Q, N=None):
    """ Integrate the ODE using classical Runge-Kutta. """
    
    assert len(Q.shape) == 2 and Q.shape[0] == Q.shape[1]
    
    # calculate a default step width
    if N is None:
        N = 2 * int(round(tau / 0.01))
        print("Steps used", N)

    A = -np.log(1 - Q).transpose()
    ind_tmp = np.arange(len(A))
    A[ind_tmp, ind_tmp ] = -A.sum(axis=0)
    
    h = tau / N

    # start value
    y = np.eye(A.shape[0])
    
    # apply trapezoid rule for the numerical integral
    solution_integral = np.zeros(y.shape)
    solution_integral[:] = 0.5 * y
    
    h_over_6 = h / 6
    h_times_05 = 0.5 * h

    for k in range(N):
        
        # Runge-Kutta: cf. Stoer/Bulirsch: "Numerische Mathematik 2", 4. Auflage, Springer, 2000
        # 
        # Phi(x, y) = 1/6 * (k1 + 2k2 + 2k3 + k4)
        #   with k1 = f(x, y) = Ay
        #        k2 = f(x + 0.5*h, y + 0.5*h*k1) = A (y + 0.5 * h * k1)
        #        k3 = f(x + 0.5*h, y + 0.5*h*k2) = A (y + 0.5 * h * k2)
        #        k4 = f(x +     h, y +     h*k3) = A (y + h * k3)
        k1 = np.dot(A, y)
        k2 = np.dot(A, y +  h_times_05 * k1)
        k3 = np.dot(A, y +  h_times_05 * k2)
        k4 = np.dot(A, y +  h * k3)
 
        y += h_over_6 * (k1 + 2 * k2 + 2 * k3 + k4)

        # add summand to integral
        solution_integral += (0.5 if k == N - 1 else 1) * y
        
    solution_integral *= h

    return y, solution_integral

In [32]:
it100, integral = integrate_general_rk(0.25, Q, N=10)
it100, integral  # check against:[0.83878624, 0.00230412, 0.15890963]

(array([[0.83878624, 0.        , 0.        ],
        [0.00230412, 1.        , 0.        ],
        [0.15890963, 0.        , 1.        ]]),
 array([[0.22926405, 0.        , 0.        ],
        [0.00029637, 0.25      , 0.        ],
        [0.02043959, 0.        , 0.25      ]]))

### Return to the original Q and compare the calculations


In [33]:
# duration is one month
tau = 1 / 12
integrate_general_rk(tau, Q_backup, N=100)

(array([[0.97312105, 0.02828162, 0.        ],
        [0.00835429, 0.93038271, 0.        ],
        [0.01852466, 0.04133567, 1.        ]]),
 array([[0.08220654, 0.00119814, 0.        ],
        [0.00035393, 0.08039595, 0.        ],
        [0.00077286, 0.00173924, 0.08333333]]))

In [34]:
integrate_general_heun(tau, Q_backup, N=400)

(array([[0.97312105, 0.02828162, 0.        ],
        [0.00835429, 0.93038271, 0.        ],
        [0.01852466, 0.04133567, 1.        ]]),
 array([[0.08220654, 0.00119814, 0.        ],
        [0.00035393, 0.08039594, 0.        ],
        [0.00077286, 0.00173925, 0.08333333]]))

In [35]:
integrate_general(tau, A)

83.33333333333333


(array([[0.97322089, 0.0281904 , 0.        ],
        [0.00832734, 0.93062041, 0.        ],
        [0.01845176, 0.04118919, 1.        ]]),
 array([[0.0818953 , 0.00117533, 0.        ],
        [0.00034719, 0.08011917, 0.        ],
        [0.00075751, 0.00170549, 0.083     ]]))